In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f) * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [3]:
# Define cubic SE6, find and classify the lines and compute the 45 tritangent planes

In [4]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
Lambda3 = {b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}
sost_SE6 = {f: -c*(5*c+e)/(c+e)}
SE6 = general_cubic.subs(Lambda3).subs(sost_SE6)

In [5]:
# Find simmetries

In [6]:
%time adm_SE6 = SE6.find_admissible_projectivities()
len(adm_SE6)

CPU times: user 516 ms, sys: 48.8 ms, total: 565 ms
Wall time: 1.43 s


48

In [7]:
%time simm_SE6 = SE6.find_simmetries(adm_SE6)
len(simm_SE6)

CPU times: user 218 ms, sys: 76.5 ms, total: 295 ms
Wall time: 389 ms


24

In [8]:
# Study how the simmetries permute the Eckardt points

In [10]:
Eck_perms = [Permutation(SE6.apply_proj_to_eck(proj[0])).to_permutation_group_element() for proj in simm_SE6]
Eck_perms_group = PermutationGroup(Eck_perms)
Eck_perms_group.order(), Eck_perms_group.structure_description()

(24, 'S4')

In [11]:
# We want to find what elements are permuted by the group S4
# These are the lines through 2 Eckardt points that are not contained in SE6

In [12]:
lines_through_2_Eck_points = []
lines_through_2_Eck_points_labels = []
lines_plucker = [line.plucker for line in SE6.lines]
eck = SE6.eckardt_points
for i in range(len(eck)):
    point1 = eck[i]
    for j in range(i+1, len(eck)):
        point2 = eck[j]
        new_line_plucker = Point(matrix([point1, point2]).minors(2))
        #check that we get a new line that does not belong to SE6
        if new_line_plucker not in lines_plucker and new_line_plucker not in [line.plucker for line in lines_through_2_Eck_points]:
            planes = get_two_planes_containing_line([point1, point2])
            lines_through_2_Eck_points_labels.append([SE6.eckardt_points_labels[i],SE6.eckardt_points_labels[j]])
            lines_through_2_Eck_points.append(Line(planes, [point1, point2], new_line_plucker))
lines_through_2_Eck_points_labels

[[['E1', 'G4', 'F14'], ['E2', 'G1', 'F12']],
 [['E1', 'G4', 'F14'], ['E2', 'G3', 'F23']],
 [['E2', 'G1', 'F12'], ['E3', 'G4', 'F34']],
 [['E2', 'G3', 'F23'], ['E3', 'G4', 'F34']]]

In [13]:
f = SymmetricGroup(4).isomorphism_to(Eck_perms_group)
gens = [f(gen) for gen in SymmetricGroup(4).gens_small()]
gens

[(1,2,4,3)(5,6), (1,5)(4,6)]

In [14]:
indices = [Eck_perms.index(gen) for gen in gens]
M4 = simm_SE6[indices[0]][0]
M2 = simm_SE6[indices[1]][0]

In [15]:
four_lines_perms = []
for proj in [M4, M2]:
    perm_indices = [lines_through_2_Eck_points.index(
        line.apply_proj(proj))+1 
                    for line in lines_through_2_Eck_points]
    perm = Permutation(perm_indices).to_permutation_group_element()
    four_lines_perms.append(perm)
four_lines_perms

[(1,2,4,3), (2,4)]

In [16]:
#check that the 4 lines are actually permuted by the simmetry group
for line in lines_through_2_Eck_points:
    print([line.apply_proj(M4) in lines_through_2_Eck_points, line.apply_proj(M2) in lines_through_2_Eck_points])

[True, True]
[True, True]
[True, True]
[True, True]


In [17]:
# Study how the simmetries permute the 27 lines

In [19]:
lines_perms = [Permutation(SE6.apply_proj_to_lines(simm[0])).to_permutation_group_element() for simm in simm_SE6]
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(24, 'S4')

In [20]:
gens_lines = [lines_perms[index] for index in indices]
[get_permuted_extended_L_set(perm) for perm in gens_lines], gens_lines

([('F23', 'E2', 'G4', 'E3', 'F12', 'G6'),
  ('G2', 'F24', 'E2', 'F23', 'F13', 'F15')],
 [(1,18,22,7)(2,10)(3,13,15,9)(4,14,8,27)(5,12,26,20)(6,11,25,21)(16,17)(23,24),
  (1,8)(3,14)(4,15)(5,16)(6,17)(9,18)(10,19)(11,20)(12,21)(22,27)(23,26)(24,25)])

In [21]:
PermutationGroup(gens_lines).is_isomorphic(Eck_perms_group)

True

In [22]:
# Find possible conditions to obtain a subfamily with a larger simmetry group

In [23]:
%time SE6.find_conditions_for_subfamilies(adm_SE6, simm_SE6)

CPU times: user 195 ms, sys: 95.8 ms, total: 291 ms
Wall time: 488 ms


[]